# MDA_HW2_106061218 李丞恩

# 一. 設定全域變數

In [1]:
from pyspark import SparkConf, SparkContext

設定點數與$\beta$值

In [2]:
vertex_num=10876 # number of vertices in the graph
beta=0.8

# 二. 會用到的mapper跟reducer，以及一些函數

首先是這個read_map(line)，它會將檔案讀成$$[(i,j)]$$的樣子，其中$i$是from end, $j$是to end。

In [3]:
def read_map(line):
    wordlist=line.split("\n")
    maplist=[]
    for item in wordlist:
        s = item.split("\t")
        for item in s:
            key=int(s[0])  # key: from_end
            value=int(s[1]) # value: To_end
        maplist.append((key,value)) # add to map
    return maplist

這個flatten2list函數會把nested list或tuple變成一維的list。

In [4]:
def flatten2list(object):
    gather = []
    for item in object:
        if isinstance(item, (list, tuple, set)):
            gather.extend(flatten2list(item))            
        else:
            gather.append(item)
    return gather

這個reducer會把所有相同key的value放一起，形成一個nested list中，所以要搭配flatten2list使用。

In [5]:
def length_reducer(x,y):
    return flatten2list((x,y))

output_reducer可以會保留所有相同key的key-value對的第一個，因此只能在確定所有相同key的key-value對都相同時使用

In [6]:
def output_reducer(x,y):
    return x

sum_reducer會在計算pagerank的迴圈裡使用，它會把所有相同key的key-value對的value加起來。

In [7]:
def sum_reducer(x,y):
    return x+y

In [8]:
def to_list_mapper(x):
    if isinstance(x[1], int):
        a=[]
        a.append(x[1])
        return (x[0],a)
    else:
        return (x[0],x[1])

接著呼叫SparkConf跟SparkContext，並停止已經運行的SparkContext，準備工作就大功告成了

In [9]:
sc.stop()

In [10]:
conf = SparkConf().setMaster("local").setAppName("wordcount") # call sparkconf
conf = SparkConf().set("spark.default.parallelism", 4)
sc = SparkContext(conf=conf) # call sparkcontext

# 三. 步驟

先建立G1跟G兩個RDD，我們的目標是利用一系列的map與reduce建立一個RDD$$G2=\bigg[j,(i, d_i, r_i)\bigg]$$其中$d_i$為$i$的out degree，$r_i$的初始值設為$1/N$。一開始，先使用read_map讀入G與G1，此時兩者是相等的。

In [11]:
G = sc.textFile("p2p-Gnutella04.txt").flatMap(read_map) # read txt file
G1 = G

利用length_reducer將G1化為$[(i,i連到的點形成的list)]$

In [12]:
G1 = G1.reduceByKey(length_reducer)
G1 = G1.map(to_list_mapper)

In [13]:
print(G.collect()[:10])
print(G1.collect()[:10])

[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10)]
[(0, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), (8, [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]), (12, [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]), (20, [50, 51, 52, 53, 54, 55, 56, 57, 58, 59]), (60, [184]), (76, [206, 207, 208, 209, 210, 211, 212, 213, 214, 215]), (80, [232, 233, 234, 235, 236, 237, 238, 239, 240]), (84, [408, 820, 2619, 2952, 4166, 4657, 4754, 5031, 6337, 6338]), (88, [84, 85, 86, 292, 293, 294, 295, 296, 297, 298]), (92, [164, 275, 276, 277, 278, 279, 280, 281, 282, 283])]


把兩者join起來，令為G2。

In [19]:
G2 = G.join(G1)

In [20]:
print(G2.collect()[:10])

[(0, (1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])), (0, (2, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])), (0, (3, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])), (0, (4, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])), (0, (5, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])), (0, (6, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])), (0, (7, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])), (0, (8, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])), (0, (9, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])), (0, (10, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))]


將G2map成$$G2=\bigg[j,(i, i連到的點形成的list所形成的長度, 初始之r_i=1/N)\bigg]=\bigg[j,(i, d_i, r_i)\bigg]$$就完成初始化了


In [21]:
G2=G2.map(lambda x: (x[1][0], [x[0], len(x[1][1]), 1/vertex_num])) # r_j^old = (j,[i, d_i, r_i])

In [22]:
print(G2.collect()[:10])

[(1, [0, 10, 9.194556822361162e-05]), (2, [0, 10, 9.194556822361162e-05]), (3, [0, 10, 9.194556822361162e-05]), (4, [0, 10, 9.194556822361162e-05]), (5, [0, 10, 9.194556822361162e-05]), (6, [0, 10, 9.194556822361162e-05]), (7, [0, 10, 9.194556822361162e-05]), (8, [0, 10, 9.194556822361162e-05]), (9, [0, 10, 9.194556822361162e-05]), (10, [0, 10, 9.194556822361162e-05])]


接下來執行迴圈，將以下動作重複20次:
1. 利用把G2給map的方法建立一個新的rdd，$$A=\bigg[(j,\beta\frac{r_i}{d_i})\bigg]$$此時$j$是可能會重複的。
2. 把A用sum_reducer給加起來，$$A=[j,\sum_{i\to j}\beta\frac{r_i}{d_i}]$$此時$j$是不會重複的。
3. 計算$$S=\sum_jr_j$$
4. 把A給map成$$A=A=\bigg[(j,r_i^{new})\bigg]$$，其中$$r_i^{new}=\sum_{i\to j}\beta\frac{r_i}{d_i}-\frac{1-S}{N})$$
5. 把G2給map成$$G2=\bigg[i,(j, d_i, r_i^{old})\bigg]$$這只是暫時的，是為了執行下一步的join並更新$r_i$
6. 把G2和A以G2.join(A)給join起來
7. 把G2給map回$$G2=\bigg[j,(i, d_i, r_i^{new})\bigg]$$
執行完20次之後，A就是我們要的結果，亦即各個頂點與其$r_i$值。

In [23]:
for i in range (1,20):
    A=G2.map(lambda x: [x[0], beta*x[1][2]/x[1][1]])
    A=A.reduceByKey(sum_reducer)
    S=A.values().sum()
    A=A.map(lambda x: [x[0], x[1]+(1-S)/vertex_num])
    G2=G2.map(lambda x: [x[1][0],[x[0],x[1][1],x[1][2]]])
    G2=G2.join(A)
    G2=G2.map(lambda x: [x[1][0][0],[x[0],x[1][0][1],x[1][1]]])

In [24]:
print(A.collect()[:10])

[[4, 0.00038399804762964836], [8, 6.698905795316284e-05], [32, 6.279908378240562e-05], [36, 8.804459800175772e-05], [40, 0.00010528568026290993], [44, 0.0002141378006685025], [48, 0.00011997612073147499], [52, 0.00015500690917152543], [56, 6.565875067850188e-05], [184, 0.00012003097423043789]]


將$A$排序後儲存到HW2_Output的資料夾中，總共生成四個檔案，手動將其改為.txt檔就可以了。

In [29]:
A = A.map(lambda x: [x[1], x[0]])
A = A.sortByKey() # sort the result
A = A.map(lambda x: [x[1], x[0]])
print(A.collect()[:10])

[[5436, 5.8253484293315e-05], [5437, 5.8253484293315e-05], [5438, 5.8253484293315e-05], [5444, 5.8253484293315e-05], [5446, 5.8253484293315e-05], [7362, 5.864240598126702e-05], [7364, 5.864240598126702e-05], [7365, 5.864240598126702e-05], [7367, 5.864240598126702e-05], [4801, 5.8738148146858956e-05]]


In [30]:
A.saveAsTextFile("HW2_Output")